In [1]:
from satpy import Scene, find_files_and_readers
from pyresample import create_area_def
from satpy.writers import get_enhanced_image
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from celluloid import Camera

from satpy import MultiScene
from satpy.multiscene import timeseries

In [2]:
# Generates a list of file names
filenames = glob('/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.*')
filenames.sort()
filenames

['/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.A2020278.0800.061.2020278191227.hdf',
 '/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.A2020278.1900.061.2020279153337.hdf',
 '/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.A2020279.0705.061.2020279213456.hdf',
 '/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.A2020279.0840.061.2020279213911.hdf',
 '/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.A2020279.0845.061.2020279213851.hdf',
 '/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.A2020279.1805.061.2020280152840.hdf',
 '/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.A2020279.1940.061.2020280152445.hdf',
 '/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.A2020280.0745.061.2020281013423.hdf',
 '/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.A2020280.1845.061.2020281152557.hdf',
 '/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.A2020280.2025.061.2020281152538.hdf',
 '/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.A2020281.0830.061.2020281190602.hdf',
 '/home/hboi-ouri/Pro

In [3]:
# 'clip' the day identifier (e.g. 'A2022060') from all file names
swats = np.unique([f.split('.')[1] for f in filenames])
swats

array(['A2020278', 'A2020279', 'A2020280', 'A2020281', 'A2020282',
       'A2020283', 'A2020284', 'A2020285', 'A2020286', 'A2020287',
       'A2020288'], dtype='<U8')

In [5]:
i=0
base = '/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.'

dayfiles = glob(base+swats[1]+'*')
print(dayfiles)
#create scene
scn = Scene(dayfiles, reader='modis_l1b')
scn.load(['10','4'])
scn.load(['true_color'])
#reproject
area_proj = {'proj': 'lcc', 'lon_0': -91., 'lat_0': 29.5, 'lat_1': 29.5, 'lat_2': 29.5}
my_area = scn['10'].attrs['area'].compute_optimal_bb_area(area_proj)
new_scn = scn.resample(my_area)
#generate RGB from true color
rgb = get_enhanced_image(new_scn['true_color'])
#extract projection
crs = new_scn['true_color'].attrs['area'].to_cartopy_crs()
#make plot
fig =  plt.figure(figsize=(6, 4), dpi=400)
ax1 = plt.subplot(projection=crs)
rgb.data.plot.imshow(rgb='bands', transform=crs, ax=ax1)
ax1.set_title('MODISA_%s' % new_scn.start_time.isoformat())

['/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.A2020279.1805.061.2020280152840.hdf', '/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.A2020279.0845.061.2020279213851.hdf', '/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.A2020279.1940.061.2020280152445.hdf', '/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.A2020279.0705.061.2020279213456.hdf', '/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.A2020279.0840.061.2020279213911.hdf']


/home/hboi-ouri/anaconda3/envs/satpy/lib/python3.8/site-packages/pyproj/crs.py:686: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj_string = self.to_proj4()
/home/hboi-ouri/anaconda3/envs/satpy/lib/python3.8/site-packages/pyresample/kd_tree.py:965: PerformanceWarning: Increasing number of chunks by factor of 12
  res = blockwise(query_no_distance, 'jik', tlons, 'ji', tlats, 'ji',
/home/hboi-ouri/anaconda3/envs/satpy/lib/python3.8/site-packages/pyresample/kd_tree.py:1136: PerformanceWarning: Increasing number of chunks by factor of 12
  res = blockwise(_my_index, dst_adims,


ImportError: Pyresample only supports converting to cartopy 0.20.0+ CRS objects. Either update cartopy or downgrade to an older version of Pyresample (<1.22.0) that supports older versions of cartopy.

In [6]:
# make them all
base = '/home/hboi-ouri/Projects/RS_Files/Delta/MYD021KM.'
# loop over unique days
for swat in swats:
    #load
    dayfiles = glob(base+swat+'*')
    scn = Scene(dayfiles, reader='modis_l1b')
    scn.load(['10','4'])
    scn.load(['true_color'])
    #project
    area_proj = {'proj': 'lcc', 'lon_0': -91., 'lat_0': 29.5, 'lat_1': 29.5, 'lat_2': 29.5}
    my_area = scn['10'].attrs['area'].compute_optimal_bb_area(area_proj)
    new_scn = scn.resample(my_area)
    #generate RGB from true color
    rgb = get_enhanced_image(new_scn['true_color'])
    #extract projection
    crs = new_scn['true_color'].attrs['area'].to_cartopy_crs()
    #make the figure
    fig =  plt.figure(figsize=(6, 4), dpi=400)
    ax1 = plt.subplot(projection=crs)
    rgb.data.plot.imshow(rgb='bands', transform=crs, ax=ax1)
    #set title with date
    ax1.set_title('MODISA_%s' % new_scn.start_time.isoformat())

/home/hboi-ouri/anaconda3/envs/satpy/lib/python3.8/site-packages/pyproj/crs.py:686: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj_string = self.to_proj4()


ImportError: Pyresample only supports converting to cartopy 0.20.0+ CRS objects. Either update cartopy or downgrade to an older version of Pyresample (<1.22.0) that supports older versions of cartopy.

In [ ]:
# projecting to a fixed area (instead of automatic optimized)
i=0
dayfiles = glob(base+swats[i]+'*')
print(dayfiles)
#create scene
scn = Scene(dayfiles, reader='modis_l1b')
scn.load(['10','4'])
scn.load(['true_color'])
#reprpject
my_area = create_area_def('my_area', {'proj': 'lcc', 'lon_0': -91., 'lat_0': 29.5, 'lat_1': 29.5, 'lat_2': 29.5},
                          width=1500, height=750,
                          area_extent=[-94, 27.5, -88, 30.5], units='degrees')
new_scn = scn.resample(my_area)
#generate RGB from true color
rgb = get_enhanced_image(new_scn['true_color'])

#extract projection and lon lat from products
crs = new_scn['true_color'].attrs['area'].to_cartopy_crs()

fig =  plt.figure(figsize=(6, 4), dpi=400)
ax1 = plt.subplot(projection=crs)
rgb.data.plot.imshow(rgb='bands', transform=crs, ax=ax1)
ax1.set_title('MODISA_Mdelta_%s' % new_scn.start_time.isoformat())

In [ ]:
# loop
for swat in swats:
    dayfiles = glob(base+swat+'*')

    print(dayfiles)
    #create scene
    scn = Scene(dayfiles, reader='modis_l1b')
    scn.load(['10','4'])
    scn.load(['true_color'])
    #reprpject
    my_area = create_area_def('my_area', {'proj': 'lcc', 'lon_0': -91., 'lat_0': 29.5, 'lat_1': 29.5, 'lat_2': 29.5},
                              width=1500, height=750,
                              area_extent=[-94, 27.5, -88, 30.5], units='degrees')
    new_scn = scn.resample(my_area)
    #generate RGB from true color
    rgb = get_enhanced_image(new_scn['true_color'])

    #extract projection and lon lat from products
    crs = new_scn['true_color'].attrs['area'].to_cartopy_crs()

    fig =  plt.figure(figsize=(6, 4), dpi=400)
    ax1 = plt.subplot(projection=crs)
    rgb.data.plot.imshow(rgb='bands', transform=crs, ax=ax1)
    ax1.set_title('MODISA_Mdelta_%s' % new_scn.start_time.isoformat())

In [ ]:
# double
i=0
dayfiles = glob(base+swats[i]+'*')
print(dayfiles)
extent=[-94, 27.5, -88, 30.5]
res = '10m'

#load scene
scn = Scene(dayfiles, reader='modis_l1b')
scn.load(['10','4'])
scn.load(['true_color'])

#reproject
my_area = create_area_def('my_area', {'proj': 'lcc', 'lon_0': -91., 'lat_0': 29.5, 'lat_1': 29.5, 'lat_2': 29.5},
                          width=1500, height=750,
                          area_extent=extent, units='degrees')
new_scn = scn.resample(my_area)

#generate RGB from true color
rgb = get_enhanced_image(new_scn['true_color'])

#calculate aCDOM
A = 0.472; B = 1.48; C = 4.64
aCDOM412 = (np.log((new_scn['10']/new_scn['4'] - A)/ B))/(-C)
aCDOM412 = aCDOM412.compute()

#extract projection and lon lat from products
crs = new_scn['true_color'].attrs['area'].to_cartopy_crs()
lons, lats = new_scn['true_color'].attrs['area'].get_lonlats()

#set up figure size and resolution
fig =  plt.figure(figsize=(6, 4), dpi=400)

#left true color
ax1 = plt.subplot(1, 2, 1, projection=crs)
rgb.data.plot.imshow(rgb='bands', transform=crs, ax=ax1)
ax1.set_title('MODISA: %s' % new_scn.start_time.isoformat())

#right aCDOM
trim = aCDOM412.max().values
ax2 = plt.subplot(1, 2, 2, projection=crs)
ax2.coastlines(res)
#this is ploting x,y, aCDOM
ax2.pcolormesh(lons, lats, aCDOM412.where(aCDOM412<=trim), transform=ccrs.PlateCarree(),
              vmin=.01, vmax=.3)
#mask the land
ax2.add_feature(cfeature.NaturalEarthFeature(category='physical', 
                                            name='land', facecolor='grey',
                                            scale=res))
ax2.set_title('aCDOM412')
#optimize spacing between plots
fig.tight_layout()
#save
fig.savefig('test2.png')


In [ ]:
# Animation

# set extent

extent=[-94, 27.5, -88, 30.5]


#create area once
my_area = create_area_def('my_area', {'proj': 'lcc', 'lon_0': -91., 'lat_0': 29.5, 'lat_1': 29.5, 'lat_2': 29.5},
                          width=1500, height=750,
                          area_extent=extent, units='degrees')
#constants
A = 0.472; B = 1.48; C = 4.64

# for animation
fig =  plt.figure(figsize=(6, 4), dpi=400)
camera = Camera(fig)

for swat in swats:
    dayfiles = glob(base+swat+'*')
    print(dayfiles)
    scn = Scene(dayfiles, reader='modis_l1b')
    scn.load(['10','4'])
    scn.load(['true_color'])

    #reproject
    new_scn = scn.resample(my_area)

    #generate RGB from true color
    rgb = get_enhanced_image(new_scn['true_color'])

    #calculate aCDOM
    aCDOM412 = (np.log((new_scn['10']/new_scn['4'] - A)/ B))/(-C)
    aCDOM412 = aCDOM412.compute()

    #extract projection and lon lat from products
    crs = new_scn['true_color'].attrs['area'].to_cartopy_crs()
    lons, lats = new_scn['true_color'].attrs['area'].get_lonlats()

    #set up figure size and resolution

    #left true color
    ax1 = plt.subplot(1, 2, 1, projection=crs)
    rgb.data.plot.imshow(rgb='bands', transform=crs, ax=ax1)
    ax1.set_title('True Color')

    #right aCDOM
    trim = aCDOM412.max().values
    ax2 = plt.subplot(1, 2, 2, projection=crs)
    #costline with specified resolution
    ax2.coastlines(res)
    #this is ploting x,y, aCDOM
    ax2.pcolormesh(lons, lats, aCDOM412.where(aCDOM412<=trim), transform=ccrs.PlateCarree(),
                  vmin=.01, vmax=.3)
    #mask the land
    ax2.add_feature(cfeature.NaturalEarthFeature(category='physical', 
                                                name='land', facecolor='grey',
                                                scale=res))
    ax2.set_title('aCDOM412')
    #optimize spacing between plots
    fig.tight_layout()
    
    #for each frame of animation
    camera.snap()
    
    #save
animation = camera.animate(interval=1000)
animation.save('animation.mp4')

In [ ]:
## IMPLEMENT THIS ONE!!!!

extent=[-94, 27.5, -88, 30.5]


#create area once
my_area = create_area_def('my_area', {'proj': 'lcc', 'lon_0': -91., 'lat_0': 29.5, 'lat_1': 29.5, 'lat_2': 29.5},
                          width=1500, height=750,
                          area_extent=extent, units='degrees')
#constants
A = 0.472; B = 1.48; C = 4.64

for swat in swats:
    dayfiles = glob(base+swat+'*')
    print(dayfiles)
    scn = Scene(dayfiles, reader='modis_l1b')
    scn.load(['10','4'])
    scn.load(['true_color'])

    #reproject
    new_scn = scn.resample(my_area)

    #generate RGB from true color
    rgb = get_enhanced_image(new_scn['true_color'])

    #calculate aCDOM
    aCDOM412 = (np.log((new_scn['10']/new_scn['4'] - A)/ B))/(-C)
    aCDOM412 = aCDOM412.compute()

    #extract projection and lon lat from products
    crs = new_scn['true_color'].attrs['area'].to_cartopy_crs()
    lons, lats = new_scn['true_color'].attrs['area'].get_lonlats()

    #set up figure size and resolution
    fig =  plt.figure(figsize=(6, 4), dpi=400)

    #left true color
    ax1 = plt.subplot(1, 2, 1, projection=crs)
    rgb.data.plot.imshow(rgb='bands', transform=crs, ax=ax1)
    ax1.set_title('MODISA: %s' % new_scn.start_time.isoformat())

    #right aCDOM
    trim = aCDOM412.max().values
    ax2 = plt.subplot(1, 2, 2, projection=crs)
    #costline with specified resolution
    ax2.coastlines(res)
    #this is ploting x,y, aCDOM
    ax2.pcolormesh(lons, lats, aCDOM412.where(aCDOM412<=trim), transform=ccrs.PlateCarree(),
                  vmin=.01, vmax=.3)
    #mask the land
    ax2.add_feature(cfeature.NaturalEarthFeature(category='physical', 
                                                name='land', facecolor='grey',
                                                scale=res))
    ax2.set_title('aCDOM412')
    #optimize spacing between plots
    fig.tight_layout()
    
    #save


In [ ]:
# test to understand the effect of resampling (interpolation)
#on aCDOM calculation

i=0
dayfiles = glob(base+swats[i]+'*')
scn = Scene(dayfiles, reader='modis_l1b')
scn.load(['10','4'])

#reproject
my_area = create_area_def('my_area', {'proj': 'lcc', 'lon_0': -91., 'lat_0': 29.5, 'lat_1': 29.5, 'lat_2': 29.5},
                          width=1500, height=750,
                          area_extent=extent, units='degrees')
new_scn = scn.resample(my_area)

#calculate aCDOM
A = 0.472; B = 1.48; C = 4.64
aCDOM412 = (np.log((scn['10']/scn['4'] - A)/ B))/(-C)
aCDOM412 = aCDOM412.compute()

aCDOM412_r = (np.log((new_scn['10']/new_scn['4'] - A)/ B))/(-C)
aCDOM412_r = aCDOM412_r.compute()


# crs1 = scn['10'].attrs['area'].to_cartopy_crs()
crs2 = new_scn['10'].attrs['area'].to_cartopy_crs()
lons1, lats1 = scn['10'].attrs['area'].get_lonlats()
lons2, lats2 = new_scn['10'].attrs['area'].get_lonlats()

fig =  plt.figure(figsize=(6, 4), dpi=400)
extent=[-94, -88, 27.5, 30.5]
ax1 = plt.subplot(1, 2, 1, projection=crs2)
ax1.coastlines(res)
ax1.set_extent(extent)
ax1.pcolormesh(lons1, lats1, aCDOM412, transform=ccrs.PlateCarree(),
              vmin=.01, vmax=.3)
ax1.add_feature(cfeature.NaturalEarthFeature(category='physical', 
                                            name='land', facecolor='grey',
                                            scale=res))
ax1.set_title('native')

ax2 = plt.subplot(1, 2, 2, projection=crs2)
ax2.coastlines(res)
ax2.set_extent(extent)
ax2.pcolormesh(lons2, lats2, aCDOM412_r, transform=ccrs.PlateCarree(),
              vmin=.01, vmax=.3)
ax2.add_feature(cfeature.NaturalEarthFeature(category='physical', 
                                            name='land', facecolor='grey',
                                            scale=res))
ax2.set_title('resampled')


fig.tight_layout()